In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May  9 11:37:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os 
import numpy as np
import pandas as pd
import json
import warnings
import logging
import gc
import random
import math
import re
import ast
from tqdm import tqdm
from typing import Optional
from datetime import datetime
import pickle
import sklearn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
import math
import copy
from typing import Optional, List

In [ ]:
pip show tqdm

Name: tqdm
Version: 4.65.0
Summary: Fast, Extensible Progress Meter
Home-page: https://tqdm.github.io
Author: 
Author-email: 
License: MPLv2.0, MIT Licences
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: cmdstanpy, gdown, hyperopt, kaggle, moviepy, nltk, panel, proglog, prophet, spacy, tensorflow-datasets, torchtext


In [ ]:
print(pickle.format_version)

4.0


In [ ]:
import platform
print(platform.python_version())

3.10.11


In [ ]:
print(sklearn.__version__)

1.2.2


In [ ]:
np.__version__

'1.22.4'

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.0 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
# from rouge_score.rouge_scorer import RougeScorer

warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, TensorDataset

from transformers import (
    BartTokenizerFast,
    AdamW
)

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using GPU")

else:
    DEVICE = torch.device("cpu")
    print("Using CPU")


from transformers import BertTokenizer, BertModel

SOURCE_MAX_LEN = 234


ACOUSTIC_DIM = 768
ACOUSTIC_MAX_LEN = 1000



VISUAL_DIM = 2048
VISUAL_MAX_LEN = 480



import random




LEARNING_RATE = 1e-4




Using GPU


In [ ]:
def set_random_seed(seed: int):
    print("Seed : {}".format(seed))

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# set_random_seed(42)
# a = np.random.randint(0, 1000)

# set_random_seed(123)
set_random_seed(994)
# set_random_seed(12345)

import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.nn import CrossEntropyLoss, MSELoss

from typing import Any, Callable, Dict, Iterable, List, Optional, Tuple, Union

from transformers.modeling_utils import PreTrainedModel, unwrap_model

from transformers import (
    BartTokenizer,
    AdamW
)
import transformers
from transformers.models.bart.configuration_bart import BartConfig

from transformers.models.bart.modeling_bart import (
    BartPretrainedModel,
    BartDecoder,
    BartLearnedPositionalEmbedding,
    BartEncoderLayer,
    shift_tokens_right,
    _make_causal_mask,
    _expand_mask
)

from transformers.modeling_outputs import (
    BaseModelOutput,
    Seq2SeqLMOutput,
    Seq2SeqModelOutput,
    Seq2SeqSequenceClassifierOutput
)

Seed : 994


In [ ]:
print(transformers.__version__)

4.28.1


In [ ]:
print(torch.__version__)

2.0.0+cu118


In [ ]:
# from transformer_encoder import TransformerEncoder

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
BART_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
            Indices of input sequence tokens in the vocabulary. Padding will be ignored by default should you provide
            it.
            Indices can be obtained using [`BartTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.
            [What are input IDs?](../glossary#input-ids)
        attention_mask (`torch.Tensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
            [What are attention masks?](../glossary#attention-mask)
        decoder_input_ids (`torch.LongTensor` of shape `(batch_size, target_sequence_length)`, *optional*):
            Indices of decoder input sequence tokens in the vocabulary.
            Indices can be obtained using [`BartTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.
            [What are decoder input IDs?](../glossary#decoder-input-ids)
            Bart uses the `eos_token_id` as the starting token for `decoder_input_ids` generation. If `past_key_values`
            is used, optionally only the last `decoder_input_ids` have to be input (see `past_key_values`).
            For translation and summarization training, `decoder_input_ids` should be provided. If no
            `decoder_input_ids` is provided, the model will create this tensor by shifting the `input_ids` to the right
            for denoising pre-training following the paper.
        decoder_attention_mask (`torch.LongTensor` of shape `(batch_size, target_sequence_length)`, *optional*):
            Default behavior: generate a tensor that ignores pad tokens in `decoder_input_ids`. Causal mask will also
            be used by default.
            If you want to change padding behavior, you should read [`modeling_bart._prepare_decoder_attention_mask`]
            and modify to your needs. See diagram 1 in [the paper](https://arxiv.org/abs/1910.13461) for more
            information on the default strategy.
        head_mask (`torch.Tensor` of shape `(encoder_layers, encoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the attention modules in the encoder. Mask values selected in `[0, 1]`:
            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.
        decoder_head_mask (`torch.Tensor` of shape `(decoder_layers, decoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the attention modules in the decoder. Mask values selected in `[0, 1]`:
            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.
        cross_attn_head_mask (`torch.Tensor` of shape `(decoder_layers, decoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the cross-attention modules in the decoder. Mask values selected in `[0,
            1]`:
            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.
        encoder_outputs (`tuple(tuple(torch.FloatTensor)`, *optional*):
            Tuple consists of (`last_hidden_state`, *optional*: `hidden_states`, *optional*: `attentions`)
            `last_hidden_state` of shape `(batch_size, sequence_length, hidden_size)`, *optional*) is a sequence of
            hidden-states at the output of the last layer of the encoder. Used in the cross-attention of the decoder.
        past_key_values (`tuple(tuple(torch.FloatTensor))`, *optional*, returned when `use_cache=True` is passed or when `config.use_cache=True`):
            Tuple of `tuple(torch.FloatTensor)` of length `config.n_layers`, with each tuple having 2 tensors of shape
            `(batch_size, num_heads, sequence_length, embed_size_per_head)`) and 2 additional tensors of shape
            `(batch_size, num_heads, encoder_sequence_length, embed_size_per_head)`.
            Contains pre-computed hidden-states (key and values in the self-attention blocks and in the cross-attention
            blocks) that can be used (see `past_key_values` input) to speed up sequential decoding.
            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`. inputs_embeds (`torch.FloatTensor` of shape
            `(batch_size, sequence_length, hidden_size)`, *optional*): Optionally, instead of passing `input_ids` you
            can choose to directly pass an embedded representation. This is useful if you want more control over how to
            convert `input_ids` indices into associated vectors than the model's internal embedding lookup matrix.
        decoder_inputs_embeds (`torch.FloatTensor` of shape `(batch_size, target_sequence_length, hidden_size)`, *optional*):
            Optionally, instead of passing `decoder_input_ids` you can choose to directly pass an embedded
            representation. If `past_key_values` is used, optionally only the last `decoder_inputs_embeds` have to be
            input (see `past_key_values`). This is useful if you want more control over how to convert
            `decoder_input_ids` indices into associated vectors than the model's internal embedding lookup matrix.
            If `decoder_input_ids` and `decoder_inputs_embeds` are both unset, `decoder_inputs_embeds` takes the value
            of `inputs_embeds`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        output_attentions (`bool`, *optional*):
            Whether or not to return the attentions tensors of all attention layers. See `attentions` under returned
            tensors for more detail.
        output_hidden_states (`bool`, *optional*):
            Whether or not to return the hidden states of all layers. See `hidden_states` under returned tensors for
            more detail.
        return_dict (`bool`, *optional*):
            Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple.
"""



In [ ]:
class ContextAwareAttention(nn.Module):

    def __init__(self,
                 dim_model : int,
                 dim_context : int,
                 dropout_rate : Optional[float] = 0.0 ):

        super(ContextAwareAttention, self).__init__()

        self.dim_model = dim_model
        self.dim_context = dim_context
        self.dropout_rate = dropout_rate
        self.attention_layer = nn.MultiheadAttention(embed_dim=self.dim_model,
                                                     num_heads = 1,
                                                     dropout = self.dropout_rate,
                                                     bias = True,
                                                    add_zero_attn=False,
                                                    batch_first=True,
                                                    device=DEVICE   
        )

        self.u_k = nn.Linear(self.dim_context, self.dim_model, bias = False)
        self.w1_k = nn.Linear(self.dim_model, 1, bias=False)
        self.w2_k = nn.Linear(self.dim_model, 1, bias=False)

        self.u_v = nn.Linear(self.dim_context, self.dim_model, bias=False)
        self.w1_v = nn.Linear(self.dim_model, 1, bias = False)
        self.w2_v = nn.Linear(self.dim_model, 1, bias = False)

    def forward(self, q, k, v, context):

        # print("Context shape : ", context.shape)
        # print("Dim context : ", self.dim_context, " : Dim model : ", self.dim_model)
        key_context = self.u_k(context)
        # print("Context shape below key context : ", key_context.shape)
        value_context = self.u_v(context)

        lambda_k = F.sigmoid(self.w1_k(k) + self.w2_k(key_context))
        lambda_v = F.sigmoid(self.w1_v(v) + self.w2_v(value_context))

        k_cap = (1-lambda_k) * k + (lambda_k) * key_context
        v_cap = (1-lambda_v) * v + (lambda_v) * value_context

        attention_output, _ = self.attention_layer(query = q,
                                                   key = k_cap,
                                                   value = v_cap)
  
        return attention_output                                     



In [ ]:
class MAF_acoustic(nn.Module):
    def __init__(self, 
                dim_model,
                dropout_rate):
        super(MAF_acoustic, self).__init__()
        self.dropout_rate = dropout_rate

        self.acoustic_context_transform = nn.Linear(ACOUSTIC_MAX_LEN, SOURCE_MAX_LEN, bias = False)
        # self.visual_context_transform = nn.Linear(VISUAL_MAX_LEN, SOURCE_MAX_LEN, bias = False)

        self.acoustic_context_attention = ContextAwareAttention(dim_model=dim_model,
                                                                dim_context=ACOUSTIC_DIM,
                                                                dropout_rate=dropout_rate)

        # self.visual_context_attention = ContextAwareAttention(dim_model=dim_model,
        #                                                     dim_context=VISUAL_DIM,
        #                                                     dropout_rate=dropout_rate)

        self.acoustic_gate = nn.Linear(2*dim_model, dim_model)
        # self.visual_gate = nn.Linear(2*dim_model, dim_model)
        self.dropout_layer = nn.Dropout(dropout_rate)
        self.final_layer_norm = nn.LayerNorm(dim_model)

    def forward(self,
                text_input,
                acoustic_context):

        # print("Acoustic context shape (A) : ", acoustic_context.shape)        

        acoustic_context = acoustic_context.permute(0,2,1)
        acoustic_context = self.acoustic_context_transform(acoustic_context.float())
        acoustic_context = acoustic_context.permute(0,2,1)

        audio_out = self.acoustic_context_attention(q=text_input,
                                                    k=text_input,
                                                    v=text_input,
                                                    context=acoustic_context)
        # print("Audio out (A) : ", audio_out.shape) 

        # print("Visual context shape : ", visual_context.shape)
        # visual_context = visual_context.permute(0,2,1)
        # visual_context = self.visual_context_transform(visual_context.float())
        # visual_context = visual_context.permute(0,2,1)
        
        # video_out = self.visual_context_attention(q=text_input,
        #                                             k=text_input,
        #                                             v=text_input,
        #                                             context=visual_context)

        # print("Video out shape : ", video_out.shape)
        # print("Text input shape : ", text_input.shape)
        weight_a = F.sigmoid(self.acoustic_gate(torch.cat([text_input, audio_out], dim=-1)))
        # weight_v = F.sigmoid(self.visual_gate(torch.cat([text_input, video_out], dim=-1)))

        # output = self.final_layer_norm(text_input + weight_a * audio_out + weight_v * video_out)

        output = self.final_layer_norm(text_input + weight_a * audio_out)

        return output

In [ ]:
class MAF_visual(nn.Module):
    def __init__(self, 
                dim_model,
                dropout_rate):
        super(MAF_visual, self).__init__()
        self.dropout_rate = dropout_rate

        # self.acoustic_context_transform = nn.Linear(ACOUSTIC_MAX_LEN, SOURCE_MAX_LEN, bias = False)
        self.visual_context_transform = nn.Linear(VISUAL_MAX_LEN, SOURCE_MAX_LEN, bias = False)

        # self.acoustic_context_attention = ContextAwareAttention(dim_model=dim_model,
        #                                                         dim_context=ACOUSTIC_DIM,
        #                                                         dropout_rate=dropout_rate)

        self.visual_context_attention = ContextAwareAttention(dim_model=dim_model,
                                                            dim_context=VISUAL_DIM,
                                                            dropout_rate=dropout_rate)

        # self.acoustic_gate = nn.Linear(2*dim_model, dim_model)
        self.visual_gate = nn.Linear(2*dim_model, dim_model)
        self.dropout_layer = nn.Dropout(dropout_rate)
        self.final_layer_norm = nn.LayerNorm(dim_model)

    def forward(self,
                text_input,
                visual_context):

        # print("Acoustic context shape (A) : ", acoustic_context.shape)        

        # acoustic_context = acoustic_context.permute(0,2,1)
        # acoustic_context = self.acoustic_context_transform(acoustic_context.float())
        # acoustic_context = acoustic_context.permute(0,2,1)

        # audio_out = self.acoustic_context_attention(q=text_input,
        #                                             k=text_input,
        #                                             v=text_input,
        #                                             context=acoustic_context)
        # print("Audio out (A) : ", audio_out.shape) 

        # print("Visual context shape : ", visual_context.shape)
        visual_context = visual_context.permute(0,2,1)
        visual_context = self.visual_context_transform(visual_context.float())
        visual_context = visual_context.permute(0,2,1)
        
        video_out = self.visual_context_attention(q=text_input,
                                                    k=text_input,
                                                    v=text_input,
                                                    context=visual_context)

        # print("Video out shape : ", video_out.shape)
        # print("Text input shape : ", text_input.shape)
        # weight_a = F.sigmoid(self.acoustic_gate(torch.cat([text_input, audio_out], dim=-1)))
        weight_v = F.sigmoid(self.visual_gate(torch.cat([text_input, video_out], dim=-1)))

        # output = self.final_layer_norm(text_input + weight_a * audio_out + weight_v * video_out)

        output = self.final_layer_norm(text_input  + weight_v * video_out)

        return output

In [ ]:
# class CustomTransformerEncoder(torch.nn.Module):
#   def __init__(self, vocab_size, hidden_dim, expand_factor):
#     super(CustomTransformerEncoder, self).__init__()


In [ ]:
# class MultiHeadSelfAttention(torch.nn.Module):
#   def __init__(self, config):
#     super(MultiHeadSelfAttention, self).__init__()

#     self.num_attention_heads = 12
#     self.attention_head_size = config.d_model//12
#     self.embed_dim = config.d_model
#     self.dropout = config.dropout

#     self.query = nn.Linear(config.d_model, config.d_model)
#     self.key = nn.Linear(config.d_model, config.d_model)
#     self.value = nn.Linear(config.d_model, config.d_model)
#     self.out_proj = nn.Linear(config.d_model, config.d_model)

#   def forward(self, hidden_states, attention_mask):
#     query_states = self.query(hidden_states)
#     key_states = self.key(hidden_states)
#     value_states = self.value(hidden_states)

#     batch_size = hidden_states.shape[0]
#     seq_len = hidden_states.shape[1]

#     query_states =  query_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()
#     key_states =  key_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()
#     value_states = value_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()

#     query_states = query_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)
#     key_states  = key_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)
#     value_states = value_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)

#     attn_weights = torch.bmm(query_states, key_states.transpose(1,2))

#     attn_weights = attn_weights.view(batch_size, self.num_attention_heads, attn_weights.shape[1], attn_weights.shape[2])
#     attn_weights = attn_weights/torch.sqrt(torch.tensor(self.attention_head_size))

#     attn_weights_masked = attn_weights.masked_fill(
#         attention_mask[:, None, None, :] == False, -1e16
#     )

#     attn_weights = torch.nn.functional.softmax(attn_weights_masked, dim = -1)

#     attn_weights = torch.nn.functional.dropout(attn_weights, p = self.dropout, training = self.training)

#     attn_weights = attn_weights.view(batch_size*self.num_attention_heads, seq_len, seq_len)


    
#     attn_output = torch.bmm(attn_weights, value_states)

#     attn_output = attn_output.view(batch_size, self.num_attention_heads, seq_len, self.attention_head_size)
#     attn_output = attn_output.transpose(1,2)

#     attn_output = attn_output.reshape(batch_size, seq_len, self.embed_dim)

#     attn_out = self.out_proj(attn_output)

#     return attn_out

In [ ]:
# def gelu(input):
#   input = torch.tensor(input)
#   return input * 0.5 * (1.0 + torch.erf(input/torch.sqrt(torch.tensor(2.0))))

In [ ]:
# class CustomEncoder(torch.nn.Module):
#   def __init__(self, config):
#     super(CustomEncoder, self).__init__()
#     self.embed_dim = config.d_model
#     self.self_attn = MultiHeadSelfAttention(config)
#     self.self_attn_layer_norm = nn.LayerNorm(self.embed_dim)
#     self.dropout = config.dropout
#     # self.activation_fn = torch.nn.ReLU()
#     self.activation_dropout = config.activation_dropout
#     self.fc1 = nn.Linear(self.embed_dim, 2*self.embed_dim)
#     self.fc2 = nn.Linear(2*self.embed_dim, self.embed_dim)
#     self.final_layer_norm = nn.LayerNorm(self.embed_dim)

#   def forward(self, hidden_states, attention_mask):
#    residual = hidden_states
#    hidden_states =  self.self_attn(hidden_states = hidden_states, attention_mask = attention_mask)
#    hidden_states = nn.functional.dropout(hidden_states, p = self.dropout, training = self.training)
#    hidden_states = residual + hidden_states
#    hidden_states = self.self_attn_layer_norm(hidden_states)

#    residual = hidden_states
#    hidden_states = gelu(self.fc1(hidden_states))
#    hidden_states = nn.functional.dropout(hidden_states, p=self.activation_dropout, training = self.training)
#    hidden_states = self.fc2(hidden_states)
#    hidden_states = nn.functional.dropout(hidden_states, p=self.activation_dropout, training = self.training)
#    hidden_states = residual + hidden_states
#    hidden_states = self.final_layer_norm(hidden_states)

#    if hidden_states.dtype == torch.float16 and (
#        torch.isinf(hidden_states).any() or torch.isnan(hidden_states).any()
#    ):
#       clamp_value = torch.finfo(hidden_states.dtype).max = -1000
#       hidden_states = torch.clamp(hidden_states, min = -clamp_value, max = clamp_value)

#    return hidden_states   

In [ ]:
# class ContextEncoder(nn.Module):
#   def __init__(self, config, num_layers):
#     super(ContextEncoder, self).__init__()

#     self.word_embeddings = nn.Embedding(config.vocab_size, config.d_model, padding_idx = config.pad_token_id)
#     self.position_embedding = nn.Embedding(500, config.d_model)
#     self.layer_norm = nn.LayerNorm(config.d_model)
#     self.context_encoder = torch.nn.ModuleList([CustomEncoder(config) for _ in range(num_layers)])
#     self.context_gate = nn.Linear(2*config.d_model, config.d_model)
    

#   def forward(self, hidden_states, context_input_ids, context_attention_mask) :
    
#     seq_len = context_input_ids.shape[1]
#     pos = torch.arange(seq_len, dtype = torch.long)
#     pos = pos.unsqueeze(0).expand_as(context_input_ids)
#     pos = pos.to(device)
#     context_state = self.word_embeddings(context_input_ids) + self.position_embedding(pos)
    
#     for layer in self.context_encoder:
#             context_state = layer(hidden_states = context_state, attention_mask  = context_attention_mask)
    

#     weight_c = F.sigmoid(self.context_gate(torch.cat([hidden_states, context_state], dim=-1)))
#     output = self.layer_norm(hidden_states + weight_c * context_state)
#     # print('transformer context shape :', output.shape)
#     return output



In [ ]:
# class ContextEncoder(nn.Module):
#   def __init__(self, config):
#     super(ContextEncoder, self).__init__()

#     self.bert = bert_model
#     self.layer_norm = nn.LayerNorm(config.d_model)
#     self.context_gate = nn.Linear(2*config.d_model, config.d_model)
    

#   def forward(self, hidden_states, context_input_ids, context_attention_mask) :
    
#     output_encoding = self.bert(context_input_ids, context_attention_mask)

#     context_state = output_encoding['last_hidden_state']
    

#     weight_c = F.sigmoid(self.context_gate(torch.cat([hidden_states, context_state], dim=-1)))
#     output = self.layer_norm(hidden_states + weight_c * context_state)
#     # print('transformer context shape :', output.shape)
#     return output



In [ ]:
class MultiModalBartEncoder(BartPretrainedModel):

    def __init__(self, config: BartConfig, embed_tokens: Optional[nn.Embedding] = None):
        super().__init__(config)

        self.dropout = config.dropout
        self.layerdrop = config.encoder_layerdrop

        embed_dim = config.d_model
        self.padding_idx = config.pad_token_id
        self.max_source_position = config.max_position_embeddings
        self.embed_scale = math.sqrt(embed_dim) if config.scale_embedding else 1.0

        if embed_tokens is not None:
            self.embed_tokens = embed_tokens
        else:
            self.embed_tokens = nn.Embedding(config.vocab_size, embed_dim, self.padding_idx)

        self.embed_positions = BartLearnedPositionalEmbedding(
            config.max_position_embeddings,
            embed_dim
        )        

        
        

        self.layers = nn.ModuleList([BartEncoderLayer(config) for _ in range(config.encoder_layers)])

        self.layernorm_embedding = nn.LayerNorm(embed_dim)

        self.init_weights()
        self.gradient_checkpointing = False

        # self.fusion_at_layer = [4]
        # self.fusion_at_layer = [3, 4]
        # self.fusion_at_layer3 = [3]
        self.fusion_at_layer4 = [4]
        self.fusion_at_layer5 = [5]

        # self.fusion_of_context = [3]
        # self.visual_transformer = TransformerEncoder(d_model = VISUAL_DIM,
        #                                              n_layers = 4,
        #                                              n_heads=8,
        #                                              d_ff=VISUAL_DIM
        #                                              )
        # self.acoustic_transformer = TransformerEncoder(d_model = ACOUSTIC_DIM,
        #                                                n_layers=4,
        #                                                n_heads=2,
        #                                                d_ff=ACOUSTIC_DIM)

        # self.MAF_layer3 = MAF(dim_model=embed_dim,
        #                      dropout_rate=0.2)
        
        self.MAF_layer4 = MAF_acoustic(dim_model=embed_dim,
                             dropout_rate=0.2)
        
        self.MAF_layer5 = MAF_visual(dim_model=embed_dim,
                             dropout_rate=0.2)
        
        # self.context_encoder = ContextEncoder(config)

        # self.classification = nn.Linear(embed_dim, 2)


        



    def forward(self,
            input_ids = None,
            attention_mask = None,
            # context_input_ids = None,
            # context_attention_mask = None,
            acoustic_input = None,
            visual_input = None,
            head_mask = None,
            inputs_embeds = None,
            output_attentions = None,
            output_hidden_states  = None,
            return_dict = None):

            # print("Input ids shape : ", input_ids.shape)
            output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions

            output_hidden_states = (
                output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
            )

            return_dict = return_dict if return_dict is not None else self.config.use_return_dict

            if input_ids is not None and inputs_embeds is not None:
                raise ValueError("You can't specify both input_ids and inputs_embeds at the same time")
            elif input_ids is not None:
                input_shape = input_ids.size()
                input_ids = input_ids.view(-1, input_shape[-1])

            elif inputs_embeds is not None:
                input_shape = inputs_embeds.size()[:-1]
            else:
                raise ValueError("You have to specify either input_ids or input_embeds")


            if inputs_embeds is None:
                # print("Input ids shape : ", input_ids.shape)
                inputs_embeds = self.embed_tokens(input_ids) * self.embed_scale

            # print("Input shape type : ", type(input_shape))
            # print("Input shape : ", input_shape)
            input_shape = torch.tensor(input_shape)
            # print("Input shape type : ", type(input_shape))
            # print("Input shape : ", input_shape)
            # print("Input shape : ", input_shape.shape)
            embed_pos = self.embed_positions(input_ids)
            # embed_pos = self.embed_positions(input_shape)


            hidden_states = inputs_embeds + embed_pos   
            hidden_states = self.layernorm_embedding(hidden_states)
            hidden_states = F.dropout(hidden_states, p = self.dropout, training=self.training)

            # print("attention mask shape 3 : ", attention_mask.shape)
            if attention_mask is not None:
                attention_mask =  _expand_mask(attention_mask, inputs_embeds.dtype)

            # print("attention mask shape 4 : ", attention_mask.shape)
            encoder_states = () if output_hidden_states else None
            all_attentions = () if output_attentions else None

            if head_mask is not None:
                assert head_mask.size()[0] == (
                    len(self.layers)
                ), f"The head mask should be specified for {len(self.layers)} layers, but it is for {head_mask.size()[0]}."

            for idx, encoder_layer in enumerate(self.layers):
                # print("============Idx : ", idx)

                # if idx in self.fusion_at_layer3:
                #     # print("Acoustic input shape (B) : ", acoustic_input)
                #     # acoustic_input = self.acoustic_transformer(acoustic_input)[-1]   
                #     # print("Acoustic input shape (C) : ", acoustic_input)

                #     # visual_input = self.visual_transformer(visual_input)[-1]
                #     # print("====Idx inside fusion at layer :", idx)
                #     hidden_states = self.MAF_layer3(text_input = hidden_states,
                #                                    acoustic_context = acoustic_input,
                #                                    visual_context = visual_input) 

                # if idx in self.fusion_of_context:
                  
                #   hidden_states = self.context_encoder(hidden_states = hidden_states, context_input_ids = context_input_ids, context_attention_mask = context_attention_mask)
                  

                if idx in self.fusion_at_layer4:
                    # print("Acoustic input shape (B) : ", acoustic_input)
                    # acoustic_input = self.acoustic_transformer(acoustic_input)[-1]   
                    # print("Acoustic input shape (C) : ", acoustic_input)

                    # visual_input = self.visual_transformer(visual_input)[-1]
                    # print("====Idx inside fusion at layer :", idx)

                   
                    
                    hidden_states = self.MAF_layer4(text_input = hidden_states,
                                                   acoustic_context = acoustic_input
                                                   )
                if idx in self.fusion_at_layer5:
                    # print("Acoustic input shape (B) : ", acoustic_input)
                    # acoustic_input = self.acoustic_transformer(acoustic_input)[-1]   
                    # print("Acoustic input shape (C) : ", acoustic_input)

                    # visual_input = self.visual_transformer(visual_input)[-1]
                    # print("====Idx inside fusion at layer :", idx)

                    hidden_states = self.MAF_layer5(text_input = hidden_states,
                                                   visual_context = visual_input)     

                if output_hidden_states:
                    encoder_states = encoder_states + (hidden_states,)

                dropout_probability = random.uniform(0,1)   

                if self.training and (dropout_probability < self.layerdrop):
                    layer_outputs = (None, None)

                else:
                    if self.gradient_checkpointing and self.training:

                        def create_custom_forward(module):
                            def custom_forward(*inputs):
                                return module(*inputs, output_attentions)

                            return custom_forward

                        layer_outputs = torch.utils.checkpoint.checkpoint(
                            create_custom_forward(encoder_layer),
                            hidden_states,
                            attention_mask,
                            (head_mask[idx] if head_mask is not None else None),
                        )        

                    else:
                        # print("Checking Attention mask shape : ", attention_mask.shape)
                        layer_outputs = encoder_layer(
                            hidden_states,
                            attention_mask,
                            layer_head_mask = (head_mask[idx] if head_mask is not None else None),
                            output_attentions = output_attentions
                        )    

                    hidden_states = layer_outputs[0]

                if output_attentions:
                    all_attentions  = all_attentions + (layer_outputs[1],)

            if output_hidden_states:
                encoder_states = encoder_states + (hidden_states,)

            if not return_dict:
                return tuple(v for v in [hidden_states, encoder_states, all_attentions] if v is not None)

            return BaseModelOutput(
                last_hidden_state=hidden_states, hidden_states=encoder_states, attentions=all_attentions
            )    

            # print("Hidden states shape : ", hidden_states)

            # cls = hidden_states.permute(1,0,2)

In [ ]:
class MultimodalBartModel(BartPretrainedModel):
    def __init__(self, config: BartConfig):
        super().__init__(config)

        padding_idx, vocab_size = config.pad_token_id, config.vocab_size
        self.shared = nn.Embedding(vocab_size, config.d_model, padding_idx)

        self.encoder = MultiModalBartEncoder(config, self.shared)
        self.decoder = BartDecoder(config, self.shared)

        self.init_weights()

    def get_input_embeddings(self):
        return self.shared

    def set_input_embeddings(self, value):
        self.shared = value
        self.encoder.embed_tokens = self.shared
        self.decoder.embed_tokens = self.shared

    def get_encoder(self):
        return self.encoder


    def get_decoder(self):
        return self.decoder

    def forward(
        self, 
        input_ids = None,
        attention_mask = None,
        # context_input_ids = None,
        # context_attention_mask = None,
        acoustic_input = None,
        visual_input = None,
        decoder_input_ids = None,
        decoder_attention_mask = None,
        head_mask = None,
        decoder_head_mask = None,
        cross_attn_head_mask = None,
        encoder_outputs = None,
        past_key_values = None,
        inputs_embeds = None,
        decoder_inputs_embeds = None,
        use_cache = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None
    ):

        if decoder_input_ids is None and decoder_inputs_embeds is None:
            decoder_input_ids = shift_tokens_right(
                input_ids, self.config.pad_token_id, self.config.decoder_start_token_id

            )                                

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )    
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # print("attention mask shape 2 : ", attention_mask.shape)

        if encoder_outputs is None:
            encoder_outputs = self.encoder(
                input_ids = input_ids,
                attention_mask = attention_mask,
                # context_input_ids = context_input_ids,
                # context_attention_mask = context_attention_mask,
                acoustic_input = acoustic_input,
                visual_input = visual_input,
                head_mask = head_mask,
                inputs_embeds = inputs_embeds,
                output_attentions = output_attentions,
                output_hidden_states = output_hidden_states,
                return_dict = return_dict
            )

        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions = encoder_outputs[2] if len(encoder_outputs) > 2 else None
            )    

        decoder_outputs = self.decoder(
            input_ids = decoder_input_ids,
            attention_mask = decoder_attention_mask,
            encoder_hidden_states = encoder_outputs[0],
            encoder_attention_mask = attention_mask,
            head_mask = decoder_head_mask,
            cross_attn_head_mask = cross_attn_head_mask,
            past_key_values = past_key_values,
            inputs_embeds = decoder_inputs_embeds,
            use_cache = use_cache,
            output_attentions = output_attentions,
            output_hidden_states = output_hidden_states,
            return_dict = return_dict
        )    

        if not return_dict:
            return decoder_outputs + encoder_outputs

        return Seq2SeqModelOutput(
            last_hidden_state=decoder_outputs.last_hidden_state,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions

        )

In [ ]:
# class MultiHeadSelfAttention(torch.nn.Module):
#   def __init__(self, config):
#     super(MultiHeadSelfAttention, self).__init__()

#     self.num_attention_heads = 12
#     self.attention_head_size = config.d_model//12
#     self.embed_dim = config.d_model
#     self.dropout = config.dropout

#     self.query = nn.Linear(config.d_model, config.d_model)
#     self.key = nn.Linear(config.d_model, config.d_model)
#     self.value = nn.Linear(config.d_model, config.d_model)
#     self.out_proj = nn.Linear(config.d_model, config.d_model)

#   def forward(self, hidden_states, attention_mask):
#     query_states = self.query(hidden_states)
#     key_states = self.key(hidden_states)
#     value_states = self.value(hidden_states)

#     batch_size = hidden_states.shape[0]
#     seq_len = hidden_states.shape[1]

#     query_states =  query_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()
#     key_states =  key_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()
#     value_states = value_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()

#     query_states = query_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)
#     key_states  = key_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)
#     value_states = value_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)

#     attn_weights = torch.bmm(query_states, key_states.transpose(1,2))

#     attn_weights = attn_weights.view(batch_size, self.num_attention_heads, attn_weights.shape[1], attn_weights.shape[2])
#     attn_weights = attn_weights/torch.sqrt(torch.tensor(self.attention_head_size))

#     attn_weights_masked = attn_weights.masked_fill(
#         attention_mask[:, None, None, :] == False, -1e16
#     )

#     attn_weights = torch.nn.functional.softmax(attn_weights_masked, dim = -1)

#     attn_weights = torch.nn.functional.dropout(attn_weights, p = self.dropout, training = self.training)

#     attn_weights = attn_weights.view(batch_size*self.num_attention_heads, seq_len, seq_len)


    
#     attn_output = torch.bmm(attn_weights, value_states)

#     attn_output = attn_output.view(batch_size, self.num_attention_heads, seq_len, self.attention_head_size)
#     attn_output = attn_output.transpose(1,2)

#     attn_output = attn_output.reshape(batch_size, seq_len, self.embed_dim)

#     attn_out = self.out_proj(attn_output)

In [ ]:
# class SelfAttention(torch.nn.Module):
#   def __init__(self):
#     super(SelfAttention, self).__init__()
#     self.num_attention_heads = 1
#     self.attention_head_size = 768
#     self.embed_dim = 768

#     self.query = torch.nn.Linear(768, 768, bias = False)
#     self.key = torch.nn.Linear(768, 768, bias = False)
#     self.value = torch.nn.Linear(768, 768, bias = False)

#     self.out_proj = torch.nn.Linear(768, 768, bias = False)

#   def forward(self, hidden_states, attention_mask)  :
#     query_states = self.query(hidden_states)
#     key_states = self.key(hidden_states)
#     value_states = self.value(hidden_states)

#     batch_size = hidden_states.shape[0]
#     seq_len = hidden_states.shape[1]

#     query_states = query_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()
#     key_states = key_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()
#     value_states = value_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()

#     attn_weights = torch.matmul(query_states, key_states.transpose(2,3))
#     attn_weights = attn_weights/torch.sqrt(torch.tensor(self.attention_head_size))

#     attn_weights_masked = attn_weights.masked_fill(
#         attention_mask[:, None, None, :] == False, -1e16
#     )

#     attn_weights = torch.nn.functional.softmax(attn_weights_masked, dim = -1)

#     attn_output = torch.matmul(attn_weights, value_states)

#     attn_output = attn_output.transpose(1,2).contiguous()

#     attn_output = attn_output.view(batch_size, seq_len, -1)

#     attn_out = self.out_proj(attn_output)

#     return attn_out

  

In [ ]:
class MultimodalBartClassification(nn.Module):
    def __init__(
        self, 
        input_dim: int,
        inned_dim: int,
        num_classes: int,
        pooler_dropout: float
    ):
        super().__init__()
        self.dense = nn.Linear(input_dim, inned_dim)
        self.dropout = nn.Dropout(p = pooler_dropout)
        self.out_proj = nn.Linear(inned_dim, num_classes)

    def forward(self, hidden_states):
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.dense(hidden_states)
        hidden_states = torch.tanh(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.out_proj(hidden_states)
        return hidden_states    




class MultimodalBartForSequenceClassification(BartPretrainedModel):
    _keys_to_ignore_on_load_missing = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight"]

    def __init__(self, config: BartConfig, **kwargs):
        super().__init__(config, **kwargs)
        self.model = MultimodalBartModel(config)
        # self.bigru = torch.nn.GRU(input_size = 768, hidden_size = 384, num_layers = 1, batch_first = True, dropout = 0.1,  bidirectional = True)
        # self.custom_attention = SelfAttention()
        self.classification_head = MultimodalBartClassification(
            config.d_model,
            config.d_model,
            2,
            config.classifier_dropout
        )
        self.model._init_weights(self.classification_head.dense)
        self.model._init_weights(self.classification_head.out_proj)

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.tensor] = None,
        # context_input_ids : torch.LongTensor = None,
        # context_attention_mask : Optional[torch.tensor] = None,
        acoustic_input = None,
        visual_input = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None
    ) -> Union[Tuple, Seq2SeqSequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        if labels is not None:
            use_cache = False

        if input_ids is None and inputs_embeds is not None:
            raise NotImplementedError(
                f"Passing input embeddings is currently not supported for {self.__class__.__name__}"
            )

        # print("attention mask shape 1 : ", attention_mask.shape )
        outputs = self.model(
            input_ids,
            attention_mask = attention_mask,
            # context_input_ids = context_input_ids,
            # context_attention_mask = context_attention_mask,
            acoustic_input = acoustic_input,
            visual_input = visual_input,
            decoder_input_ids = decoder_input_ids,
            decoder_attention_mask = decoder_attention_mask,
            head_mask = head_mask,
            decoder_head_mask = decoder_head_mask,
            cross_attn_head_mask = cross_attn_head_mask,
            encoder_outputs = encoder_outputs,
            inputs_embeds = inputs_embeds,
            decoder_inputs_embeds = decoder_inputs_embeds,
            use_cache = use_cache,
            output_attentions = output_attentions,
            output_hidden_states = output_hidden_states,
            return_dict = return_dict

        )    

        hidden_states = outputs[0]

        eos_mask = input_ids.eq(self.config.eos_token_id).to(hidden_states.device)

        if len(torch.unique_consecutive(eos_mask.sum(1))) > 1:
            raise ValueError("All examples must have same number of <eos> tokens")

        sentence_representation = hidden_states[eos_mask, :].view(hidden_states.size(0), -1, hidden_states.size(-1))[
            :, -1, :
        ]    

        logits = self.classification_head(sentence_representation)

        # print("hidden_states shape : ", hidden_states.shape)

        # gru_out, _ = self.bigru(hidden_states)

        # print("gru out shape : ", gru_out.shape)

      

        # attn_output = self.custom_attention(gru_out, attention_mask)

        # print("attention_output shape : ", attn_output.shape)

        # logits = self.classification_head(attn_output[:, -1, :])
        # logits = self.classification_head(gru_out[:, -1, :])

        

        loss = None

        loss_fct = CrossEntropyLoss()
        # print("logits shape : ", logits.shape)
        loss = loss_fct(logits.view(-1, 2), labels.view(-1))
        # print("Loss : ", loss)
        if not return_dict:
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output


        return Seq2SeqSequenceClassifierOutput(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            decoder_hidden_states=outputs.decoder_hidden_states,
            decoder_attentions = outputs.decoder_attentions,
            cross_attentions=outputs.cross_attentions,
            encoder_last_hidden_state=outputs.encoder_last_hidden_state,
            encoder_hidden_states=outputs.encoder_hidden_states,
            encoder_attentions=outputs.encoder_attentions
        ) 

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/train_dataset_dlog_id.p', 'rb') as f:
  train_dlog_id = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/valid_dataset_dlog_id.p', 'rb') as f:
  valid_dlog_id = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/test_dataset_dlog_id.p', 'rb') as f:
  test_dlog_id = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/train_audio_data.p', 'rb') as f:
  train_audio_data = pickle.load(f)


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/valid_audio_data.p', 'rb') as f:
  valid_audio_data = pickle.load(f)


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/test_audio_data.p', 'rb') as f:
  test_audio_data = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/train_video_data.p', 'rb') as f:
  train_video_data = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/valid_video_data.p', 'rb') as f:
  valid_video_data = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/data/test_video_data.p', 'rb') as f:
  test_video_data = pickle.load(f)

In [ ]:
len(test_video_data)

75

In [ ]:
tp = torch.ones(4,5)
tp

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [ ]:
torch.zeros(5 - tp.shape[0], 5)

tensor([[0., 0., 0., 0., 0.]])

In [ ]:
torch.cat([tp, torch.zeros(5 - tp.shape[0], 5)])

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.]])

In [ ]:
def pad_seq(tensor, dim, max_len):
  if max_len > tensor.shape[0] :
    return torch.cat([tensor, torch.zeros(max_len - tensor.shape[0], dim)])
  else:
    return tensor[:max_len]  

In [ ]:
ACOUSTIC_DIM

768

In [ ]:
train_audio_data = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = ACOUSTIC_DIM,
                                                   max_len = ACOUSTIC_MAX_LEN)
                                                  for a in train_audio_data], 0)
train_audio_data.shape

torch.Size([540, 1000, 768])

In [ ]:
valid_audio_data = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = ACOUSTIC_DIM,
                                                   max_len = ACOUSTIC_MAX_LEN)
                                                  for a in valid_audio_data], 0)
valid_audio_data.shape

torch.Size([75, 1000, 768])

In [ ]:
test_audio_data = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = ACOUSTIC_DIM,
                                                   max_len = ACOUSTIC_MAX_LEN)
                                                  for a in test_audio_data], 0)
test_audio_data.shape

torch.Size([75, 1000, 768])

In [ ]:
VISUAL_DIM

2048

In [ ]:
VISUAL_MAX_LEN

480

In [ ]:
train_video_data = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = VISUAL_DIM,
                                                   max_len = VISUAL_MAX_LEN)
                                                  for a in train_video_data], 0)
train_video_data.shape

torch.Size([540, 480, 2048])

In [ ]:
valid_video_data = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = VISUAL_DIM,
                                                   max_len = VISUAL_MAX_LEN)
                                                  for a in valid_video_data], 0)
valid_video_data.shape

torch.Size([75, 480, 2048])

In [ ]:
test_video_data = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = VISUAL_DIM,
                                                   max_len = VISUAL_MAX_LEN)
                                                  for a in test_video_data], 0)
test_video_data.shape

torch.Size([75, 480, 2048])

In [ ]:
model = MultimodalBartForSequenceClassification.from_pretrained("facebook/bart-base")
print(model)

tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
print("Tokenizer : ", tokenizer)

num_param = sum(p.numel() for p in model.parameters())
print("Total parameters : ", num_param/1e6)

Some weights of MultimodalBartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.dense.bias', 'classification_head.out_proj.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MultimodalBartForSequenceClassification(
  (model): MultimodalBartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): MultiModalBartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_laye

Tokenizer :  BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Total parameters :  151.779026


In [ ]:
cnt = 0
for name, param in model.named_parameters():
    print("Count : ", cnt, " name : ", name)
    cnt+=1

Count :  0  name :  model.shared.weight
Count :  1  name :  model.encoder.embed_positions.weight
Count :  2  name :  model.encoder.layers.0.self_attn.k_proj.weight
Count :  3  name :  model.encoder.layers.0.self_attn.k_proj.bias
Count :  4  name :  model.encoder.layers.0.self_attn.v_proj.weight
Count :  5  name :  model.encoder.layers.0.self_attn.v_proj.bias
Count :  6  name :  model.encoder.layers.0.self_attn.q_proj.weight
Count :  7  name :  model.encoder.layers.0.self_attn.q_proj.bias
Count :  8  name :  model.encoder.layers.0.self_attn.out_proj.weight
Count :  9  name :  model.encoder.layers.0.self_attn.out_proj.bias
Count :  10  name :  model.encoder.layers.0.self_attn_layer_norm.weight
Count :  11  name :  model.encoder.layers.0.self_attn_layer_norm.bias
Count :  12  name :  model.encoder.layers.0.fc1.weight
Count :  13  name :  model.encoder.layers.0.fc1.bias
Count :  14  name :  model.encoder.layers.0.fc2.weight
Count :  15  name :  model.encoder.layers.0.fc2.bias
Count :  16  

In [ ]:
cnt = 0
for name, param in model.named_parameters():
    # print("Count : ", cnt, " name : ", name)
    # if(cnt>=100 and cnt<=177):
    # if(cnt>=66 and cnt<=151):
    # if(cnt>=100 and cnt<=179):
    # if(cnt>=100 and cnt<=155):
    # if(cnt>=100 and cnt<=189):
    # if(cnt>=100 and cnt<=225):
    # if(cnt>=100 and cnt<=177):
    if(cnt>=100 and cnt<=129):
    # if(cnt>=100 and cnt<=125):
        param.requires_grad = True
        print("Count : ", cnt, " name : ", name)
    # elif(cnt>=337):
    # elif(cnt>=343):
    # elif(cnt>=315):
    # elif(cnt>=329 and cnt<=332):
    #     param.requires_grad = True
    #     print("Count : ", cnt, " name : ", name)
    elif(cnt>=289):
    # elif(cnt>=385):
    # elif(cnt>=337):  
    # elif(cnt>=311):
    # elif(cnt>=285):  
        param.requires_grad = True
        print("Count : ", cnt, " name : ", name)

    else:
        param.requires_grad = False    
    cnt+=1
        

num_param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainanable parameters : ", num_param/1e6)

Count :  100  name :  model.encoder.MAF_layer4.acoustic_context_transform.weight
Count :  101  name :  model.encoder.MAF_layer4.acoustic_context_attention.attention_layer.in_proj_weight
Count :  102  name :  model.encoder.MAF_layer4.acoustic_context_attention.attention_layer.in_proj_bias
Count :  103  name :  model.encoder.MAF_layer4.acoustic_context_attention.attention_layer.out_proj.weight
Count :  104  name :  model.encoder.MAF_layer4.acoustic_context_attention.attention_layer.out_proj.bias
Count :  105  name :  model.encoder.MAF_layer4.acoustic_context_attention.u_k.weight
Count :  106  name :  model.encoder.MAF_layer4.acoustic_context_attention.w1_k.weight
Count :  107  name :  model.encoder.MAF_layer4.acoustic_context_attention.w2_k.weight
Count :  108  name :  model.encoder.MAF_layer4.acoustic_context_attention.u_v.weight
Count :  109  name :  model.encoder.MAF_layer4.acoustic_context_attention.w1_v.weight
Count :  110  name :  model.encoder.MAF_layer4.acoustic_context_attention

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/35/data/train_text_data.p", "rb") as f:
    train_text_data = pickle.load(f)

len(train_text_data)

540

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/35/data/valid_text_data.p", "rb") as f:
    valid_text_data = pickle.load(f)

len(valid_text_data)

75

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/35/data/test_text_data.p", "rb") as f:
    test_text_data = pickle.load(f)

len(test_text_data)

75

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/35/data/train_label_data.p", "rb") as f:
    train_label_data = pickle.load(f)

len(train_label_data)

540

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/35/data/valid_label_data.p", "rb") as f:
    valid_label_data = pickle.load(f)

len(valid_label_data)

75

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/35/data/test_label_data.p", "rb") as f:
    test_label_data = pickle.load(f)

len(test_label_data)

75

In [ ]:
print(type(train_audio_data))
print(train_audio_data.shape)

<class 'torch.Tensor'>
torch.Size([540, 1000, 768])


In [ ]:
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [ ]:
p = {
        'additional_special_tokens' : ['[CONTEXT]', '[UTTERANCE]']
    }

tokenizer.add_special_tokens(p)

tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['[CONTEXT]', '[UTTERANCE]']}, clean_up_tokenization_spaces=True)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50267, 768)

In [ ]:
def prepare_dataset(text_data):

            total_context_length = 0        

            dialog = []
            labels = []
            sample_utterances = []
            for i in range(len(text_data)):
                data_point = text_data[i]

                example_speaker = data_point['speaker']
                example_utterance = data_point['utterance']
                temp_label = int(data_point['sarcasm'])
                sample_utterances.append(data_point)

                # example_dialog = '[CONTEXT] '
                # example_dialog = '[TARGET] '
                example_dialog = '[CONTEXT] '

                count = 0
                context_length = len(data_point['context_speakers'])

                context_list = []

                total_context_length += len(data_point['context_speakers'])
                for speaker, utterance in list(zip(data_point['context_speakers'], data_point['context'])):
                    temp_context = speaker.upper() + " : " + utterance + " | "
                    context_list.append(temp_context)
                    
                    # example_dialog = example_dialog + speaker.upper() + " : " + utterance + " | "

                # example_dialog += context_list[-1:]
                
                for i, item in enumerate(context_list):
                  if(i>=0):
                    example_dialog += item
                    
                  if(i==context_length-1):
                    break  

                print("Context : ", context_list)
                print("Truncated context : ", example_dialog)

                example_dialog = example_dialog + ' [UTTERANCE] ' + example_speaker + " : " + example_utterance + " | "
                # example_dialog = example_dialog + example_speaker + " : " + example_utterance
                # example_dialog = example_dialog + example_speaker + " : " + example_utterance 
                # print(example_dialog)
                example_dialog = re.sub(' +', ' ', example_dialog)

                dialog.append(example_dialog)
                labels.append(temp_label)

                # print(data_point)
                # print(temp_label)

            # df = pd.DataFrame(dialog, columns=['dialog'])

            labels = torch.tensor(labels, dtype=torch.long)

            

            enc = tokenizer(dialog, max_length = SOURCE_MAX_LEN, padding = 'max_length', truncation = True)

            # df['audio_features'] = acoustic_data
            # df['visual_features'] = visual_data

            return torch.tensor(enc['input_ids'], dtype=torch.long), torch.tensor(enc['attention_mask'], dtype=torch.bool), total_context_length


In [ ]:
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['[CONTEXT]', '[UTTERANCE]']}, clean_up_tokenization_spaces=True)

In [ ]:
len(train_text_data)

540

In [ ]:
train_text_data

[{'utterance': "It's just a privilege to watch your mind at work.",
  'speaker': 'SHELDON',
  'context': ['I never would have identified the fingerprints of string theory in the aftermath of the Big Bang.',
   "My apologies. What's your plan?"],
  'context_speakers': ['LEONARD', 'SHELDON'],
  'show': 'BBT',
  'sarcasm': True},
 {'utterance': "If you're compiling a mix CD for a double suicide. Oh, I hope that scratching post is for you.",
  'speaker': 'SHELDON',
  'context': ['Oh, good Lord.', "God, that's a good song."],
  'context_speakers': ['SHELDON', 'LEONARD'],
  'show': 'BBT',
  'sarcasm': True},
 {'utterance': '- Now?! - No, after my tongue has swollen to the size of a brisket!',
  'speaker': 'HOWARD',
  'context': ['Take that!',
   "What's the matter?",
   'This granola bar has peanuts in it!',
   'Oh, my God. Why did you eat it?',
   "I don't know. It was just there!",
   'If I had a gun there, would you have shot yourself?',
   "Don't yell at me! I've got to go to the emergen

In [ ]:
train_text_input_ids, train_text_attention_mask, total_context_length1 = prepare_dataset(train_text_data)


Context :  ['LEONARD : I never would have identified the fingerprints of string theory in the aftermath of the Big Bang. | ', "SHELDON : My apologies. What's your plan? | "]
Truncated context :  [CONTEXT] LEONARD : I never would have identified the fingerprints of string theory in the aftermath of the Big Bang. | SHELDON : My apologies. What's your plan? | 
Context :  ['SHELDON : Oh, good Lord. | ', "LEONARD : God, that's a good song. | "]
Truncated context :  [CONTEXT] SHELDON : Oh, good Lord. | LEONARD : God, that's a good song. | 
Context :  ['LEONARD : Take that! | ', "LEONARD : What's the matter? | ", 'HOWARD : This granola bar has peanuts in it! | ', 'LEONARD : Oh, my God. Why did you eat it? | ', "HOWARD : I don't know. It was just there! | ", 'LEONARD : If I had a gun there, would you have shot yourself? | ', "HOWARD : Don't yell at me! I've got to go to the emergency room! | "]
Truncated context :  [CONTEXT] LEONARD : Take that! | LEONARD : What's the matter? | HOWARD : This g

In [ ]:
valid_text_input_ids, valid_text_attention_mask, total_context_length2 = prepare_dataset(valid_text_data)
test_text_input_ids, test_text_attention_mask, total_context_length3 = prepare_dataset(test_text_data)


Context :  ['LEONARD : Okay, why? | ']
Truncated context :  [CONTEXT] LEONARD : Okay, why? | 
Context :  ["BERNADETTE : You heard what he said about his parents. \nIt was the little things they kept bottled up. \nI don't want that to happen to us. | ", "HOWARD : How can I convince you it won't? | ", "BERNADETTE : Well, is there anything about me you're keeping inside? | ", "HOWARD : I'm not answering that. \nIt's a trap. | ", "BERNADETTE : So there<i> are</i> things you don't like. | "]
Truncated context :  [CONTEXT] BERNADETTE : You heard what he said about his parents. 
It was the little things they kept bottled up. 
I don't want that to happen to us. | HOWARD : How can I convince you it won't? | BERNADETTE : Well, is there anything about me you're keeping inside? | HOWARD : I'm not answering that. 
It's a trap. | BERNADETTE : So there<i> are</i> things you don't like. | 
Context :  ["HOWARD : And I love that you're strong and independent. | ", 'BERNADETTE : And yet, I still love whe

In [ ]:
print("TYPE : train_text_input_ids : ", type(train_text_input_ids))

TYPE : train_text_input_ids :  <class 'torch.Tensor'>


In [ ]:
total_context_length1

1754

In [ ]:
total_context_length2

262

In [ ]:
total_context_length3

245

In [ ]:
net_length = total_context_length1 + total_context_length2 + total_context_length3
net_length

2261

In [ ]:
net_length/690


3.2768115942028984

In [ ]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>', '<mask>', '[CONTEXT]', '[UTTERANCE]']

In [ ]:
tokenizer.all_special_tokens


['<s>', '</s>', '<unk>', '<pad>', '<mask>', '[CONTEXT]', '[UTTERANCE]']

In [ ]:
print(tokenizer.all_special_ids)

[0, 2, 3, 1, 50264, 50265, 50266]


In [ ]:
train_label_data

[['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 1.0],
 ['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 5.0],
 ['Others', 'True', 0.0, 8.0],
 ['Others', 'True', -1.0, '4,6'],
 ['Others', 'True', -1.0, '4,1'],
 ['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, '4,6'],
 ['Others', 'True', 1.0, 7.0],
 ['Others', 'True', -1.0, 1.0],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', -1.0, 6.0],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 6.0],
 ['Others', 'True', -1.0, '1,6'],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', -1.0, 1.0],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', 1.0, 8.0],
 ['Others', 'True', -1.0, 8.0],
 ['Others', 'True', 1.0, 8.0],
 ['Others', 'True', -1.0, 6.0],
 ['O

In [ ]:
valid_label_data[:5]

[['Others', 'True', -1.0, 6.0],
 ['Others', 'True', -1.0, 4.0],
 ['Others', 'True', -1.0, 1.0],
 ['Others', 'True', -1.0, 1.0],
 ['Others', 'True', -1.0, 3.0]]

In [ ]:
test_label_data[:5]

[['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 1.0],
 ['Others', 'True', -1.0, 4.0],
 ['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 4.0]]

In [ ]:
test_label_data

[['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 1.0],
 ['Others', 'True', -1.0, 4.0],
 ['Others', 'True', -1.0, 9.0],
 ['Others', 'True', -1.0, 4.0],
 ['Agreement', 'True', -1.0, 4.0],
 ['Agreement', 'True', 1.0, 7.0],
 ['Answer', 'True', 1.0, 7.0],
 ['Answer', 'True', -1.0, 4.0],
 ['Answer', 'True', -1.0, 4.0],
 ['Statement Opinion', 'True', -1.0, 9.0],
 ['Statement Opinion', 'True', -1.0, 9.0],
 ['Disagreement', 'True', -1.0, 6.0],
 ['Question', 'True', -1.0, 1.0],
 ['Statement Non Opinion', 'True', 1.0, 8.0],
 ['Others', 'False', 1.0, 8.0],
 ['Others', 'False', 0.0, 8.0],
 ['Others', 'False', 1.0, 7.0],
 ['Others', 'False', 1.0, 8.0],
 ['Others', 'False', 1.0, 5.0],
 ['Statement Opinion', 'False', -1.0, 4.0],
 ['Statement Opinion', 'False', 1.0, 7.0],
 ['Answer', 'False', -1.0, 4.0],
 ['Answer', 'False', 1.0, 7.0],
 ['Answer', 'False', -1.0, 4.0],
 ['Question', 'False', -1.0, 4.0],
 ['Question', 'False', -1.0, 5.0],
 ['Acknowledge', 'False', -1.0, 8.0],
 ['Command', 'False

In [ ]:
label_id_dict = {'False' : 0,
                 'True' : 1}

In [ ]:
train_sarcasm_label = []
valid_sarcasm_label = []
test_sarcasm_label = []

for j in train_label_data:
  # print()
  temp_sarcasm1 = label_id_dict[j[1]]
  train_sarcasm_label.append(temp_sarcasm1)

for k in valid_label_data:
  temp_sarcasm2 = label_id_dict[k[1]]
  valid_sarcasm_label.append(temp_sarcasm2)

for u in test_label_data:
  temp_sarcasm3 = label_id_dict[u[1]]
  test_sarcasm_label.append(temp_sarcasm3)

In [ ]:
train_sarcasm_label = torch.tensor(train_sarcasm_label)
valid_sarcasm_label = torch.tensor(valid_sarcasm_label)
test_sarcasm_label = torch.tensor(test_sarcasm_label)

print(train_sarcasm_label.shape)
print(valid_sarcasm_label.shape)
print(test_sarcasm_label.shape)

torch.Size([540])
torch.Size([75])
torch.Size([75])


In [ ]:
valid_sarcasm_label

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1])

In [ ]:
test_sarcasm_label

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1])

In [ ]:
class MultimodalSarcasmDataset(Dataset):
    # def __init__(self, utterance_input_ids, utterance_attention_mask, context_input_ids, context_attention_mask, acoustic_data, visual_data, labels):
    def __init__(self, utterance_input_ids, utterance_attention_mask, acoustic_data, visual_data, labels):

        self.utterance_input_ids = utterance_input_ids
        self.utterance_attention_mask = utterance_attention_mask
        # self.context_input_ids = context_input_ids
        # self.context_attention_mask = context_attention_mask
        # self.context_attention_mask
        self.acoustic_data = acoustic_data
        self.visual_data = visual_data
        self.labels = labels

    def __len__(self):
        return len(self.utterance_input_ids)

    def __getitem__(self, idx):
        # return self.utterance_input_ids[idx], self.utterance_attention_mask[idx], self.context_input_ids[idx], self.context_attention_mask[idx], self.acoustic_data[idx], self.visual_data[idx], self.labels[idx]
        return self.utterance_input_ids[idx], self.utterance_attention_mask[idx],  self.acoustic_data[idx], self.visual_data[idx], self.labels[idx]



In [ ]:
# train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask, train_context_input_ids, train_context_attention_mask,train_audio_broadcast_utterance, train_image_broadcast_utterance, train_ground_truth), batch_size=32, shuffle = True)
# valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask, valid_context_input_ids, valid_context_attention_mask, valid_audio_broadcast_utterance, valid_image_broadcast_utterance, valid_ground_truth), batch_size = 32, shuffle = False)
# test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask, test_context_input_ids, test_context_attention_mask, test_audio_broadcast_utterance, test_image_broadcast_utterance, test_ground_truth), batch_size=32, shuffle = False)

# train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask, train_context_input_ids, train_context_attention_mask, train_audio_data_utterance, train_image_data_utterance, train_ground_truth), batch_size=32, shuffle = True)
# # valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask, valid_context_input_ids, valid_context_attention_mask,   valid_audio_data, valid_image_data, valid_ground_truth), batch_size = 32, shuffle = False)
# test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask, test_context_input_ids, test_context_attention_mask,  test_audio_data_utterance1, test_image_data_utterance1, test_ground_truth), batch_size=32, shuffle = False)

train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask,  train_audio_data, train_video_data, train_sarcasm_label), batch_size=32, shuffle = True)
valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask,    valid_audio_data, valid_video_data, valid_sarcasm_label), batch_size = 32, shuffle = False)
test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask,   test_audio_data, test_video_data, test_sarcasm_label), batch_size=32, shuffle = False)


print(test_loader)

# print(train_loader)

In [ ]:
len(train_loader.dataset)

540

In [ ]:
len(valid_loader.dataset)

75

In [ ]:
len(test_loader.dataset)

75

In [ ]:
# base_params_list = []
# new_params_list = []
# weight_decay = 1e-4
# for name, param in model.named_parameters():
#         if "classification_head" or "MAF_layer" in name:
#             new_params_list.append(param)
#         else:
#             base_params_list.append(param)
            
#         optimizer = torch.optim.AdamW(
#         [
#             {'params': base_params_list,'lr': 5e-6, 'weight_decay': weight_decay},
#             {'params': new_params_list,'lr': LEARNING_RATE, 'weight_decay': weight_decay}            
#         ],
#         lr=5e-6,
#         weight_decay=weight_decay
#     )

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# class label_smoothing_loss(torch.nn.Module):
#   def __init__(self, num_classes, smoothing = 0.1):
#     super(label_smoothing_loss, self).__init__()
#     eps = smoothing/num_classes
#     self.negative = eps
#     self.positive = (1-smoothing) + eps

#   def forward(self, pred, target):
#     pred = pred.log_softmax(dim = -1)
#     true_dist = torch.zeros_like(pred)
#     true_dist.fill_(self.negative)
#     target_data = target.unsqueeze(dim = -1)
#     true_dist.scatter_(dim = -1, index  = target_data, src = torch.tensor(self.positive).expand(target_data.shape).cuda())
#     return torch.sum(-true_dist * pred, dim = -1).mean()   

In [ ]:
# criterion = label_smoothing_loss(num_classes = 2, smoothing = 0.1)

In [ ]:
model.config

BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_ty

In [ ]:
DEVICE

device(type='cuda')

In [ ]:
model = model.to(DEVICE)
model


MultimodalBartForSequenceClassification(
  (model): MultimodalBartModel(
    (shared): Embedding(50267, 768)
    (encoder): MultiModalBartEncoder(
      (embed_tokens): Embedding(50267, 768)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps

In [ ]:
def train_epoch(model, data_loader):
      model.train()
      epoch_train_loss = 0.0
    
      
      for step, batch in enumerate(tqdm(data_loader, desc = 'Training Iteration')):
        # for i, t in enumerate(batch):
        #     print("Inside hello")
        #     print(i, " : ", type(t))
        batch = tuple(t.to(DEVICE) for t in batch)
        # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
        input_ids, attention_mask,  acoustic_input, visual_input, labels = batch
        optimizer.zero_grad()
        # print("Input ids shape : ", input_ids.shape)
        # print("Input ids shape : ", input_ids.shape)
        outputs = model(input_ids = input_ids,
                        attention_mask = attention_mask,
                        # context_input_ids = context_input_ids,
                        # context_attention_mask = context_attention_mask,
                        acoustic_input = acoustic_input,
                        visual_input = visual_input,
                        labels = labels)

        loss = outputs['loss']
        # logits = outputs['logits']
        # loss = criterion(logits, labels)
        epoch_train_loss += loss.item()

        # print("Batch wise loss : ", epoch_train_loss)

        loss.backward()
        optimizer.step()

    

      print("Epoch train loss : ", epoch_train_loss)

In [ ]:
def valid_epoch(model, data_loader):
  model.eval()
  predictions = []
  gold = []

  valid_loss = 0.0
  with torch.no_grad():
    for step, batch in enumerate(tqdm(data_loader)):
      batch = tuple(t.to(DEVICE) for t in batch)
      # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
      input_ids, attention_mask,  acoustic_input, visual_input, labels = batch

      outputs = model(input_ids = input_ids,
                            attention_mask = attention_mask,
                            # context_input_ids = context_input_ids,
                            # context_attention_mask = context_attention_mask,
                            acoustic_input = acoustic_input,
                            visual_input = visual_input,
                            labels = labels)
      
      logits = outputs['logits']
      loss = outputs['loss']
      # loss = criterion(logits, labels)

      valid_loss += loss.item()



      pred = logits.argmax(dim = -1)

      predictions.extend(pred.tolist())
      gold.extend(labels.tolist())

  return valid_loss, predictions, gold



In [ ]:


def test_epoch(model, data_loader):
    model.eval()
    predictions = []
    gold = []

    correct = 0
    with torch.no_grad():
        for step, batch in enumerate(tqdm(data_loader)):
            batch = tuple(t.to(DEVICE) for t in batch)
            # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
            input_ids, attention_mask,  acoustic_input, visual_input, labels = batch

            outputs = model(input_ids = input_ids,
                            attention_mask = attention_mask,
                            # context_input_ids = context_input_ids,
                            # context_attention_mask = context_attention_mask,
                      
                            acoustic_input = acoustic_input,
                            visual_input = visual_input,
                            labels = labels)

            logits = outputs['logits']

            pred = logits.argmax(dim = -1)

            predictions.extend(pred.tolist())

            gold.extend(labels.tolist())

            correct += int((pred == labels).sum())

    return correct/len(data_loader.dataset), predictions, gold 

In [ ]:
class EarlyStopping:
  def __init__(self, patience, min_delta):
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.min_validation = np.inf

  def early_stop(self, valid_loss):
    if valid_loss < self.min_validation:
      self.min_validation = valid_loss
      self.counter = 0
    elif valid_loss > (self.min_validation + self.min_delta):
      self.counter += 1
      if self.counter >= self.patience:
        return True
    return False          

In [ ]:
early_stopper = EarlyStopping(patience = 8, min_delta = 0.1)
early_stopper

In [ ]:

def train_and_validation(model, train_loader, valid_loader):
  # lowest_loss = 1e6
  best_f1 = 0.0
  # min_loss = 1e6
  # for t in range(20):
  for epoch in range(20):
    # epoch = t + 20
    print("\n=============Epoch : ", epoch)
    train_epoch(model, train_loader)
    valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

    # if early_stopper.early_stop(valid_loss):
    #   break

    print("Length of predictions : ", len(valid_pred))
    print("Length of gold : ", len(valid_gold))
    print("Valid loss : ", valid_loss)
    print("\n Valid Accuracy : ", accuracy_score(valid_gold, valid_pred))
    print("\n Valid Precision : ", precision_score(valid_gold, valid_pred, average = 'weighted'))
    print("\n Valid Recall : ", recall_score(valid_gold, valid_pred, average = 'weighted'))
    print("\nValid F1 score : ", f1_score(valid_gold, valid_pred, average = 'weighted')) 

    
    curr_f1 = f1_score(valid_gold, valid_pred, average = 'weighted')

    curr_loss = valid_loss
    # if((curr_f1 > best_f1) and (epoch>=4)):
    if(curr_f1 > best_f1):
    # if(curr_loss < min_loss):    
    # if(curr_loss < lowest_loss):    
      best_f1 = curr_f1
      # min_loss = curr_loss
      # print("Valid pred : ", valid_pred)
      # print('valid_gold : ', valid_gold)
      # torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/35/saved_model/balanced_sarcasm/try_version/best_model_epoch_'+str(epoch)+'_best_f1_'+str(int(best_f1*100))+'.pt')
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/35/saved_model/ablation/sarcasm_only/best_model_epoch_'+str(epoch)+'_best_f1_'+str(int(best_f1*100))+'.pt')

      print("model saved\n")




In [ ]:
# train_and_validation(model, train_loader, test_loader)
train_and_validation(model, train_loader, valid_loader)


=============Epoch :  0


Training Iteration: 100%|██████████| 17/17 [00:09<00:00,  1.79it/s]


Epoch train loss :  12.517319321632385


100%|██████████| 3/3 [00:00<00:00,  6.12it/s]


Length of predictions :  75
Length of gold :  75
Valid loss :  2.099656879901886

 Valid Accuracy :  0.48

 Valid Precision :  0.24986301369863015

 Valid Recall :  0.48

Valid F1 score :  0.3286486486486486
model saved


=============Epoch :  1


...

Length of predictions :  75
Length of gold :  75
Valid loss :  8.559698820114136

 Valid Accuracy :  0.6133333333333333

 Valid Precision :  0.6619774011299435

 Valid Recall :  0.6133333333333333

Valid F1 score :  0.5785852298515205

=============Epoch :  19


Training Iteration: 100%|██████████| 17/17 [00:06<00:00,  2.67it/s]


Epoch train loss :  0.4025974655523896


100%|██████████| 3/3 [00:00<00:00,  5.92it/s]

Length of predictions :  75
Length of gold :  75
Valid loss :  8.324825286865234

 Valid Accuracy :  0.6

 Valid Precision :  0.6197530864197531

 Valid Recall :  0.6

Valid F1 score :  0.5796101949025487


In [ ]:


# path = '/content/drive/MyDrive/Colab Notebooks/35/saved_model/balanced_sarcasm/best_model_epoch_7_best_f1_75.pt'
path = '/content/drive/MyDrive/Colab Notebooks/35/saved_model/balanced_sarcasm/try_version/best_model_epoch_7_best_f1_72.pt'

In [ ]:
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
acc, train_pred, train_gold = test_epoch(model, train_loader)

print(acc)

print("\nAccuracy : ", accuracy_score(train_gold, train_pred))
print("\nPrecision : ", precision_score(train_gold, train_pred, average = 'weighted'))
print("\nRecall : ", recall_score(train_gold, train_pred, average = 'weighted'))
print("\nF1 score : ", f1_score(train_gold, train_pred, average = 'weighted'))
        




print('\nConfusion Matrix : \n', confusion_matrix(train_gold, train_pred))

100%|██████████| 17/17 [00:06<00:00,  2.52it/s]


0.8314814814814815

Accuracy :  0.8314814814814815

Precision :  0.8315154776641522

Recall :  0.8314814814814815

F1 score :  0.8314728120901491

Confusion Matrix : 
 [[222  47]
 [ 44 227]]


In [ ]:
acc, valid_pred, valid_gold = test_epoch(model, valid_loader)

print(acc)

print("\nAccuracy : ", accuracy_score(valid_gold, valid_pred))
print("\nPrecision : ", precision_score(valid_gold, valid_pred, average = 'weighted'))
print("\nRecall : ", recall_score(valid_gold, valid_pred, average = 'weighted'))
print("\nF1 score : ", f1_score(valid_gold, valid_pred, average = 'weighted'))
        




print('\nConfusion Matrix : \n', confusion_matrix(valid_gold, valid_pred))

100%|██████████| 3/3 [00:00<00:00,  6.03it/s]

0.72

Accuracy :  0.72

Precision :  0.7202560455192035

Recall :  0.72

F1 score :  0.7200000000000001

Confusion Matrix : 
 [[27 11]
 [10 27]]


In [ ]:


acc, test_pred, test_gold = test_epoch(model, test_loader)

print(acc)

print("\nAccuracy : ", accuracy_score(test_gold, test_pred))
print("\nPrecision : ", precision_score(test_gold, test_pred, average = 'weighted'))
print("\nRecall : ", recall_score(test_gold, test_pred, average = 'weighted'))
print("\nF1 score : ", f1_score(test_gold, test_pred, average = 'weighted'))
        




print('\nConfusion Matrix : ', confusion_matrix(test_gold, test_pred))

sar = 0

for j in range(len(test_pred)):
  if((test_pred[j]==1) and (test_gold[j] == 1)):
    sar+=1


print("Sarcastic count : ", sar)


100%|██████████| 3/3 [00:00<00:00,  5.99it/s]

0.8

Accuracy :  0.8

Precision :  0.8103703703703704

Recall :  0.8

F1 score :  0.7979859737457292

Confusion Matrix :  [[34  4]
 [11 26]]
Sarcastic count :  26


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/saved_model/output_file/test_gold_sarcasm_single_task.p', 'wb') as f:
  pickle.dump(test_gold, f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/35/saved_model/output_file/test_pred_sarcasm_single_task.p', 'wb') as f:
  pickle.dump(test_pred, f)